In [93]:
# Install necessary libraries
!pip install yfinance mplfinance pandas

print("Libraries installed successfully.")

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --- 1. THE ORIGINAL "HOLY GRAIL" STRATEGY ---
tickers_to_test = ["GBPUSD=X", "GBPJPY=X", "GBPAUD=X", "EURGBP=X"]
days_to_backtest = 180
timeframe = "1h"
initial_capital = 1000.0

risk_per_trade_pct = 0.5
risk_reward_ratio = 1.5

# --- 2. THE LOOP FOR MULTI-PAIR TESTING ---
for ticker in tickers_to_test:
    print("\n" + "#"*70)
    print(f"###   RUNNING THE ORIGINAL 'HOLY GRAIL' TEST FOR: {ticker}   ###")
    print("#"*70 + "\n")

    # --- DATA FETCHING ---
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_to_backtest)
    print(f"Fetching {timeframe} data for {ticker} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")
    data = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

    if data.empty:
        print(f"No data downloaded for {ticker}.")
        continue
    else:
        data.dropna(inplace=True)
        print("Data fetched successfully.")

        # --- PURE, UNFILTERED BACKTESTER (REBUILT WITH NUMPY) ---
        trades = []
        equity_curve = [initial_capital]
        active_trade = None

        # THE DEFINITIVE FIX: Convert all columns to NumPy arrays BEFORE the loop
        opens = data['Open'].to_numpy(dtype=float)
        highs = data['High'].to_numpy(dtype=float)
        lows = data['Low'].to_numpy(dtype=float)
        closes = data['Close'].to_numpy(dtype=float)
        index_values = data.index.to_numpy()

        for i in range(1, len(data)): # Start from 1 to safely reference i-1
            current_balance = equity_curve[-1]

            if active_trade:
                pnl = 0
                if active_trade['type'] == 'buy' and highs[i] >= active_trade['take_profit']: pnl = active_trade['risk_amount'] * risk_reward_ratio
                elif active_trade['type'] == 'buy' and lows[i] <= active_trade['stop_loss']: pnl = -active_trade['risk_amount']
                elif active_trade['type'] == 'sell' and lows[i] <= active_trade['take_profit']: pnl = active_trade['risk_amount'] * risk_reward_ratio
                elif active_trade['type'] == 'sell' and highs[i] >= active_trade['stop_loss']: pnl = -active_trade['risk_amount']

                if pnl != 0:
                    current_balance += pnl
                    active_trade['pnl'] = pnl
                    trades.append(active_trade)
                    active_trade = None

            # --- PURE NUMPY ENTRY LOGIC (NO FILTERS) ---
            if not active_trade:
                # Buy Signal: Previous candle was bearish, current candle is a bullish rejection
                if closes[i-1] < opens[i-1] and (closes[i] > closes[i-1]):
                    entry_price = closes[i]
                    stop_loss = lows[i-1]
                    if entry_price > stop_loss:
                        risk_amount = current_balance * (risk_per_trade_pct / 100.0)
                        take_profit = entry_price + ((entry_price - stop_loss) * risk_reward_ratio)
                        active_trade = {'type': 'buy', 'entry_time': index_values[i], 'risk_amount': risk_amount, 'stop_loss': stop_loss, 'take_profit': take_profit}

                # Sell Signal: Previous candle was bullish, current candle is a bearish rejection
                elif closes[i-1] > opens[i-1] and (closes[i] < closes[i-1]):
                    entry_price = closes[i]
                    stop_loss = highs[i-1]
                    if stop_loss > entry_price:
                        risk_amount = current_balance * (risk_per_trade_pct / 100.0)
                        take_profit = entry_price - ((stop_loss - entry_price) * risk_reward_ratio)
                        active_trade = {'type': 'sell', 'entry_time': index_values[i], 'risk_amount': risk_amount, 'stop_loss': stop_loss, 'take_profit': take_profit}

            equity_curve.append(current_balance)

        # --- PERFORMANCE RESULTS ---
        final_balance = float(equity_curve[-1])
        ec_series = pd.Series(equity_curve)
        drawdown_series = (ec_series - ec_series.cummax()) / ec_series.cummax()
        max_drawdown = float(abs(drawdown_series.min()))

        title = f"THE ORIGINAL 'HOLY GRAIL' on {ticker}"
        print("\n" + "="*50 + f"\n       RESULTS FOR {title}\n" + "="*50)

        if not trades:
            print("No trades were executed.")
        else:
            trades_df = pd.DataFrame(trades)
            wins = trades_df[trades_df['pnl'] > 0]; losses = trades_df[trades_df['pnl'] < 0]
            win_rate = float(len(wins) / len(trades_df) * 100) if not trades_df.empty else 0.0
            total_pnl = final_balance - initial_capital
            avg_win = float(wins['pnl'].mean()) if not wins.empty else 0.0
            avg_loss = float(abs(losses['pnl'].mean())) if not losses.empty else 0.0

            print(f"Period Tested:   {days_to_backtest} days\nInitial Balance: ${initial_capital:.2f}\nFinal Balance:   ${final_balance:.2f}\nTotal Net PnL:   ${total_pnl:.2f} ({total_pnl/initial_capital*100:.2f}%)")
            print("-" * 50 + "\n--- PROP FIRM METRICS ---\n" + f"Max Drawdown:    {max_drawdown * 100:.2f}%")
            print("-" * 50 + "\n--- PERFORMANCE METRICS ---\n" + f"Total Trades:    {len(trades_df)}\nWin Rate:        {win_rate:.2f}%\nAverage Win:     ${avg_win:.2f}\nAverage Loss:    ${avg_loss:.2f}")
            print("="*50)

/tmp/ipython-input-2133036809.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)
[*********************100%***********************]  1 of 1 completed


######################################################################
###   RUNNING THE ORIGINAL 'HOLY GRAIL' TEST FOR: GBPUSD=X   ###
######################################################################

Fetching 1h data for GBPUSD=X from 2025-04-21 to 2025-10-18...



/tmp/ipython-input-2133036809.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)


Data fetched successfully.

       RESULTS FOR THE ORIGINAL 'HOLY GRAIL' on GBPUSD=X
Period Tested:   180 days
Initial Balance: $1000.00
Final Balance:   $1250.43
Total Net PnL:   $250.43 (25.04%)
--------------------------------------------------
--- PROP FIRM METRICS ---
Max Drawdown:    9.38%
--------------------------------------------------
--- PERFORMANCE METRICS ---
Total Trades:    466
Win Rate:        43.99%
Average Win:     $8.50
Average Loss:    $5.72

######################################################################
###   RUNNING THE ORIGINAL 'HOLY GRAIL' TEST FOR: GBPJPY=X   ###
######################################################################

Fetching 1h data for GBPJPY=X from 2025-04-21 to 2025-10-18...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2133036809.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)


Data fetched successfully.

       RESULTS FOR THE ORIGINAL 'HOLY GRAIL' on GBPJPY=X
Period Tested:   180 days
Initial Balance: $1000.00
Final Balance:   $1092.56
Total Net PnL:   $92.56 (9.26%)
--------------------------------------------------
--- PROP FIRM METRICS ---
Max Drawdown:    10.06%
--------------------------------------------------
--- PERFORMANCE METRICS ---
Total Trades:    473
Win Rate:        41.65%
Average Win:     $7.75
Average Loss:    $5.19

######################################################################
###   RUNNING THE ORIGINAL 'HOLY GRAIL' TEST FOR: GBPAUD=X   ###
######################################################################

Fetching 1h data for GBPAUD=X from 2025-04-21 to 2025-10-18...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2133036809.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)


Data fetched successfully.

       RESULTS FOR THE ORIGINAL 'HOLY GRAIL' on GBPAUD=X
Period Tested:   180 days
Initial Balance: $1000.00
Final Balance:   $1267.12
Total Net PnL:   $267.12 (26.71%)
--------------------------------------------------
--- PROP FIRM METRICS ---
Max Drawdown:    8.89%
--------------------------------------------------
--- PERFORMANCE METRICS ---
Total Trades:    558
Win Rate:        43.55%
Average Win:     $8.39
Average Loss:    $5.62

######################################################################
###   RUNNING THE ORIGINAL 'HOLY GRAIL' TEST FOR: EURGBP=X   ###
######################################################################

Fetching 1h data for EURGBP=X from 2025-04-21 to 2025-10-18...


[*********************100%***********************]  1 of 1 completed

Data fetched successfully.

       RESULTS FOR THE ORIGINAL 'HOLY GRAIL' on EURGBP=X
Period Tested:   180 days
Initial Balance: $1000.00
Final Balance:   $1280.10
Total Net PnL:   $280.10 (28.01%)
--------------------------------------------------
--- PROP FIRM METRICS ---
Max Drawdown:    10.95%
--------------------------------------------------
--- PERFORMANCE METRICS ---
Total Trades:    414
Win Rate:        44.93%
Average Win:     $8.40
Average Loss:    $5.62
